## Example of plotting RunChains

As described here: https://github.com/wandb/runchain

In [ ]:
import weave
from weave.legacy.weave.ecosystem import wandb

In [ ]:
# big project, maybe 41 deep
#chain = wandb.run_chain('shawn/branchtest_400_200_1000/2ji8emz2')

# big project, 61 deep
#chain = wandb.run_chain('shawn/branchtest_400_200_1000/sf0vvene')

# big project, 141 deep
#chain = wandb.run_chain('shawn/branchtest_400_200_1000/hja8k932')

# 3 deep, 200 metrics, 10k steps each (the default, this is fast for CI)
chain = wandb.run_chain('shawn/run_chain_20_200_10000/h4vz7n29')

# 14 deep, 200 metrics, 10k steps each
#chain = wandb.run_chain('shawn/run_chain_20_200_10000/6wdyzlq9')

In [ ]:
# display the chain information
chain

In [ ]:
# or fetch the chain object
weave.use(chain)

In [ ]:
weave.use(chain.segments.count())

In [ ]:
# view the chain's metrics table
chain.history()

In [ ]:
# Display plots from a RunChain. You need to do some resizing to seem them all currently.

def plot_metric(run, metric_name):
    return weave.legacy.weave.panels.Plot(
            run.history(),
            series=[
                # Use two plot series, one for the avg line, and one for
                # min/max area. We could reduce duplication here a bit more
                # by updating the Plot API to allow common features of series
                # to be factored out.
                weave.legacy.weave.panels.Series(
                    run.history(),
                    select_functions={
                        'x': lambda row, index: index.bin(
                                weave.legacy.weave.ops.numbers_bins_equal(
                                    weave.legacy.weave.ops.make_list(a=0, b=run.history().count()), 250))["start"],
                        'y': lambda row: row[metric_name].avg()
                    },
                    groupby_dims=['x'],
                    constants=weave.legacy.weave.panels.PlotConstants(
                        mark='line'
                    )
                ),
                weave.legacy.weave.panels.Series(
                    run.history(),
                    select_functions={
                        'x': lambda row, index: index.bin(
                                weave.legacy.weave.ops.numbers_bins_equal(
                                    weave.legacy.weave.ops.make_list(a=0, b=run.history().count()), 250))["start"],
                        'y': lambda row: row[metric_name].min(),
                        'y2': lambda row: row[metric_name].max(),
                    },
                    groupby_dims=['x'],
                    constants=weave.legacy.weave.panels.PlotConstants(
                        mark='area'
                    )
                )
            ],
        )

board = weave.legacy.weave.panels.Board(
    vars={
        'run_chain': chain,
    },
    panels=[
        weave.legacy.weave.panels.BoardPanel(
            lambda run_chain: weave.legacy.weave.panels.Group(
                layoutMode='flow',
                #gridConfig=bank.flow_nxn(2, 3),
                items={k: plot_metric(run_chain, k) for k in ['metric' + str(i) for i in range(10)]}
            ),
            layout=weave.legacy.weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=12)
        )
    ]
)
board